In [ ]:
import nltk
from nltk.corpus import brown
from nltk.tokenize import sent_tokenize, word_tokenize
import math


In [ ]:
from tqdm import tqdm


from corpus_raws import corpus_processing

corpus = corpus_processing('brown')

In [ ]:
print(type(corpus)) 
corpus[:100]

We want to creat an array or dictionary to compute the co-occurence matrix as in the GloVe Algorithm/paper.

In [ ]:

window_size = 2
from collections import defaultdict

# Create co-occurrence matrix
co_occurrence_matrix = defaultdict(lambda: defaultdict(int))


for i, target_word in enumerate(corpus):
    start = max(0, i - window_size)
    end = min(len(corpus), i + window_size + 1)
    context_words = corpus[start:end]
    context_words.remove(target_word)
    for context_word in context_words:
        #print(target_word)
        #print(context_word)
        co_occurrence_matrix[target_word][context_word] += 1

print(type(co_occurrence_matrix))



  



In [ ]:
import numpy as np

# Optionally, convert the matrix to a numpy array for further analysis
unique_words = list(set(corpus))
matrix_size = len(unique_words)
matrix = np.zeros((matrix_size, matrix_size))

word_to_index = {word: i for i, word in enumerate(unique_words)}
print(type(co_occurrence_matrix))

for target_word, context_words in co_occurrence_matrix.items():
    for context_word, count in context_words.items():
        matrix[word_to_index[target_word]][word_to_index[context_word]] = count

print("\nCo-occurrence matrix as a numpy array:")
print(type(matrix))

In [ ]:
import numpy as np

# Assuming you have a matrix named 'matrix'
matrix = np.array([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9]])

# Calculate the norm of each row
row_norms = np.linalg.norm(matrix, ord=1, axis=1, keepdims=True)
print(row_norms)

# Divide each entry by its row's norm
normalized_matrix = matrix / row_norms

print("Original Matrix:")
print(type(matrix))

print("\nNormalized Matrix:")
print(normalized_matrix)


In [ ]:
all_zeros = not np.any(matrix)
print(all_zeros)

np.all(matrix==0)

In [ ]:
# Set hyperparameters
embedding_size = 50
learning_rate = 0.1
epochs = 100

# Initialize word vectors
word_vectors = np.eye(len(unique_words), embedding_size)

In [ ]:
import torch
import numpy as np

word_vectors = torch.tensor(word_vectors, requires_grad=True)
learning_rate = 0.01
epochs = 25
batch_size = 64  # Adjust as needed

optimizer = torch.optim.SGD([word_vectors], lr=learning_rate)

for epoch in range(epochs):
    total_loss = 0

    for target_word, context_words in co_occurrence_matrix.items():
        for context_word, count in context_words.items():
            i = word_to_index[target_word]
            j = word_to_index[context_word]

            dot_product = torch.dot(word_vectors[i], word_vectors[j])
            log_co_occurrence = np.log(count)

            loss = (dot_product - log_co_occurrence)**2
            total_loss += loss.item()

            if (i + 1) % batch_size == 0:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {total_loss}")


In [ ]:
import torch

# Assuming you already have tensors word_vectors, co_occurrence_matrix, etc.

word_vectors = torch.tensor(word_vectors, requires_grad=True)

learning_rate = 0.01
epochs = 100

for epoch in range(epochs):
    total_loss = 0

    for target_word, context_words in co_occurrence_matrix.items():
        for context_word, count in context_words.items():
            i = word_to_index[target_word]
            j = word_to_index[context_word]

            # Dot product and log co-occurrence count
            dot_product = torch.dot(word_vectors[i], word_vectors[j])
            log_co_occurrence = np.log(count)  

            # Loss function
            loss = (dot_product - log_co_occurrence)**2
            total_loss += loss.item()

            # Update word vectors using gradient descent
            loss.backward()  # Backpropagation
            with torch.no_grad():
                word_vectors -= learning_rate * word_vectors.grad
                word_vectors.grad.zero_()  # Zero gradients for the next iteration

    print(f"Epoch {epoch + 1}, Loss: {total_loss}")


In [ ]:
# Trained word vectors
print("Trained Word Vectors:")
print(word_vectors)

In [ ]:
#import torch

#co_occurrence_tensor = torch.Tensor([[co_occurrence_matrix[target_word].get(context_word, 0) for context_word in unique_words] for target_word in unique_words])
#print(co_occurrence_tensor)
#print(unique_words)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

# Assuming co_occurrence_matrix is already defined

# Set hyperparameters
embedding_size = 50
learning_rate = 0.05
epochs = 100

# Convert co-occurrence matrix to torch tensors
unique_words = list(set(word for target_word, context_words in co_occurrence_matrix.items() for word in [target_word] + list(context_words.keys())))
word_to_index = {word: idx for idx, word in enumerate(unique_words)}

print(f"Target words: {unique_words}")

print(f"Context words: {unique_words}")

#co_occurrence_tensor = torch.Tensor([[co_occurrence_matrix[target_word].get(context_word, 0) for context_word in unique_words] for target_word in unique_words])
co_occurrence_tensor = torch.Tensor([
    [
        co_occurrence_matrix[target_word].get(context_word, 0) 
        for context_word in unique_words[:-1]  # Exclude the last word
    ] + [
        co_occurrence_matrix[target_word].get(unique_words[-1], 0)  # Use the last word separately
    ] 
    for target_word in unique_words
])

class GloVeModel(nn.Module):
    def __init__(self, vocab_size, embedding_size):
        super(GloVeModel, self).__init__()
        self.target_embeddings = nn.Embedding(vocab_size, embedding_size)
        self.context_embeddings = nn.Embedding(vocab_size, embedding_size)
        self.bias_target = nn.Embedding(vocab_size, 1)
        self.bias_context = nn.Embedding(vocab_size, 1)

    def forward(self, i, j):
        target_embedding = self.target_embeddings(i)
        context_embedding = self.context_embeddings(j)
        bias_target = self.bias_target(i).squeeze()
        bias_context = self.bias_context(j).squeeze()
        dot_product = torch.sum(target_embedding * context_embedding, dim=1)
        return dot_product + bias_target + bias_context

# Initialize model and optimizer
glove_model = GloVeModel(len(unique_words), embedding_size)
optimizer = optim.Adam(glove_model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    optimizer.zero_grad()
    i, j = torch.nonzero(co_occurrence_tensor, as_tuple=True)
    targets = torch.log(co_occurrence_tensor[i, j])
    predictions = glove_model(i, j)
    loss = nn.functional.mse_loss(predictions, targets)
    loss.backward()
    optimizer.step()
    
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# Trained word vectors
trained_word_vectors = glove_model.target_embeddings.weight.detach().numpy()
print("Trained Word Vectors:")
print(trained_word_vectors)


In [ ]:
import torch

co_occurrence_tensor = torch.Tensor([
    [
        co_occurrence_matrix[target_word].get(context_word, 0) 
        for context_word in unique_words[:-1]  # Exclude the last word
    ] + [
        co_occurrence_matrix[target_word].get(unique_words[-1], 0)  # Use the last word separately
    ] 
    for target_word in unique_words
])

print(co_occurrence_tensor)
print(unique_words)


In [ ]:
co_occurrence_tensor = torch.Tensor([[co_occurrence_matrix[target_word].get(context_word, 0) for context_word in unique_words] for target_word in unique_words])
print(co_occurrence_tensor)
print(unique_words)

In [2]:
Text = "The quick brown fox jumps over the tree. The quick brown fox jumps over the tree."

Text = Text.lower()
Text = Text.replace(".","")
tokens = Text.split()

print(tokens)

['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'tree', 'the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'tree']


In [3]:

for i, target_word in enumerate(tokens):
    print(f"This is the {i}-th position corresponding to the word: {target_word}")


This is the 0-th position corresponding to the word: the
This is the 1-th position corresponding to the word: quick
This is the 2-th position corresponding to the word: brown
This is the 3-th position corresponding to the word: fox
This is the 4-th position corresponding to the word: jumps
This is the 5-th position corresponding to the word: over
This is the 6-th position corresponding to the word: the
This is the 7-th position corresponding to the word: tree
This is the 8-th position corresponding to the word: the
This is the 9-th position corresponding to the word: quick
This is the 10-th position corresponding to the word: brown
This is the 11-th position corresponding to the word: fox
This is the 12-th position corresponding to the word: jumps
This is the 13-th position corresponding to the word: over
This is the 14-th position corresponding to the word: the
This is the 15-th position corresponding to the word: tree


In [10]:
from Cat_Dist_GloVe import (
    compute_context_words,
    compute_co_occurrence_matrix,
    defaultdict_to_ndarray,
    compute_GloVe_probability_matrix,
    probability_to_distance,
)
print(tokens)
print(compute_context_words(tokens,0,"the", 2))
#print(compute_context_words(tokens,9,"quick", 2))
compute_co_occurrence_matrix(tokens, 2)


['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'tree', 'the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'tree']
['quick', 'brown']


Computing GloVe Co-occurrence Matrix: 16it [00:00, 98544.59it/s]


defaultdict(<function Cat_Dist_GloVe.compute_co_occurrence_matrix.<locals>.<lambda>()>,
            {'the': defaultdict(int,
                         {'quick': 2,
                          'brown': 2,
                          'jumps': 2,
                          'over': 2,
                          'tree': 3,
                          'the': 2}),
             'quick': defaultdict(int,
                         {'the': 2, 'brown': 2, 'fox': 2, 'tree': 1}),
             'brown': defaultdict(int,
                         {'the': 2, 'quick': 2, 'fox': 2, 'jumps': 2}),
             'fox': defaultdict(int,
                         {'quick': 2, 'brown': 2, 'jumps': 2, 'over': 2}),
             'jumps': defaultdict(int,
                         {'brown': 2, 'fox': 2, 'over': 2, 'the': 2}),
             'over': defaultdict(int,
                         {'fox': 2, 'jumps': 2, 'the': 2, 'tree': 2}),
             'tree': defaultdict(int, {'over': 2, 'the': 3, 'quick': 1})})

In [7]:
sorted(set(tokens))

['brown', 'fox', 'jumps', 'over', 'quick', 'the', 'tree']

In [8]:
my_dict = compute_co_occurrence_matrix(tokens, 2)

X = defaultdict_to_ndarray(my_dict)
print(X)
len(my_dict)

Computing GloVe Co-occurrence Matrix: 16it [00:00, 75573.05it/s]

[[2 0 2 2 2 2 3]
 [2 2 0 0 0 2 1]
 [0 2 2 0 2 2 0]
 [2 0 2 2 2 0 0]
 [2 2 0 2 0 2 0]
 [0 2 2 0 0 2 2]
 [0 0 0 2 1 3 0]]


7

In [12]:

prob_array = compute_GloVe_probability_matrix(X)
print(prob_array)

[[0.15384615 0.         0.15384615 0.15384615 0.15384615 0.15384615
  0.23076923]
 [0.28571429 0.28571429 0.         0.         0.         0.28571429
  0.14285714]
 [0.         0.25       0.25       0.         0.25       0.25
  0.        ]
 [0.25       0.         0.25       0.25       0.25       0.
  0.        ]
 [0.25       0.25       0.         0.25       0.         0.25
  0.        ]
 [0.         0.25       0.25       0.         0.         0.25
  0.25      ]
 [0.         0.         0.         0.33333333 0.16666667 0.5
  0.        ]]


In [13]:
dist_array = probability_to_distance(prob_array)
print(dist_array)

[[1.87180218        inf 1.87180218 1.87180218 1.87180218 1.87180218
  1.46633707]
 [1.25276297 1.25276297        inf        inf        inf 1.25276297
  1.94591015]
 [       inf 1.38629436 1.38629436        inf 1.38629436 1.38629436
         inf]
 [1.38629436        inf 1.38629436 1.38629436 1.38629436        inf
         inf]
 [1.38629436 1.38629436        inf 1.38629436        inf 1.38629436
         inf]
 [       inf 1.38629436 1.38629436        inf        inf 1.38629436
  1.38629436]
 [       inf        inf        inf 1.09861229 1.79175947 0.69314718
         inf]]


/mnt/4070544070543F3A/Trabajo/Doctorado/Word Embeddings/Divergence Code/Cat_Dist_GloVe.py:70: RuntimeWarning: divide by zero encountered in log
  #---Maybe reference the paper for inspiration---#


In [16]:
import numpy as np

dimension = 1


from sklearn.manifold import MDS
mdsD = MDS(n_components=dimension, 
              metric=True, 
              normalized_stress=False,   
              dissimilarity='precomputed'
              )

mdsP = MDS(n_components=dimension, 
              metric=False, 
              normalized_stress=True,   
              dissimilarity='precomputed'
              )

D = np.array([[0, 2, 4],
             [2, 0, 2],
             [4, 2, 0]])

P = np.array([[1, 0.5, 0.25],
             [0.5, 1, 0.5],
             [0.25, 0.5, 1]])

mds_embedding_D = mdsD.fit_transform(D)
mds_embedding_P = mdsP.fit_transform(P)

stress_D = mdsD.stress_
stress_P = mdsP.stress_

print(mds_embedding_D)
print(mds_embedding_P)

print(stress_D)
print(stress_P)


[[-2.]
 [ 0.]
 [ 2.]]
[[-0.28446415]
 [ 0.57521624]
 [-0.29494405]]
0.0
0.005335514101335329


In [17]:
#d = D.shape[0]
#D[range(d),range(d)] = 1

exp_P = np.exp(-D)

print(D)
print(exp_P)

[[0 2 4]
 [2 0 2]
 [4 2 0]]
[[1.         0.13533528 0.01831564]
 [0.13533528 1.         0.13533528]
 [0.01831564 0.13533528 1.        ]]


In [22]:


exp_P = np.exp(-D)
print(exp_P)
#d = D.shape[0]
#D[range(d),range(d)] = 1

mds_embedding_exP = mdsP.fit(exp_P).embedding_
stress_exP = mdsP.stress_

print(mds_embedding_exP)
print(stress_exP)

[[1.         0.13533528 0.01831564]
 [0.13533528 1.         0.13533528]
 [0.01831564 0.13533528 1.        ]]
[[-0.28460756]
 [ 0.57528985]
 [-0.29473211]]
0.005154604131453216


In [23]:
nmds_embedding_D = mdsD.fit_transform(exp_P)
print(nmds_embedding_D)

[[-0.03900655]
 [ 0.09022352]
 [-0.05121697]]


In [20]:
print(exp_P)
ln_X = -np.log(exp_P)
print(ln_X)

mds_embedding_exP = mdsD.fit_transform(ln_X)
stress_ln =  mdsD.stress_

print(mds_embedding_exP)
print(stress_exP)

[[1.         0.13533528 0.01831564]
 [0.13533528 1.         0.13533528]
 [0.01831564 0.13533528 1.        ]]
[[-0.  2.  4.]
 [ 2. -0.  2.]
 [ 4.  2. -0.]]
[[ 2.]
 [ 0.]
 [-2.]]
0.005004649928775297
